In [67]:
import requests
import pymorphy2

In [68]:
morph = pymorphy2.MorphAnalyzer()

In [252]:
s = "В Москве в воскресенье днем начнется сильный снегопад его принесет циклон Сара сообщил РИА Новости ведущий сотрудник центра погоды Фобос Евгений Тишковец"

In [253]:
s.split(' ')

['В',
 'Москве',
 'в',
 'воскресенье',
 'днем',
 'начнется',
 'сильный',
 'снегопад',
 'его',
 'принесет',
 'циклон',
 'Сара',
 'сообщил',
 'РИА',
 'Новости',
 'ведущий',
 'сотрудник',
 'центра',
 'погоды',
 'Фобос',
 'Евгений',
 'Тишковец']

In [254]:
MODEL = 'ruscorpora_upos_cbow_300_20_2019'
FORMAT = 'csv'

def api_neighbor(m, w, f):
    neighbors = []
    url = '/'.join(['http://rusvectores.org', m, w, 'api', f]) + '/'
    r = requests.get(url=url, stream=True)
    
    if len(r.text.split('\n')) == 1:
        return 'none'
    
    orig_word = r.text.split('\n')[1]
    pos = orig_word.split('_')[1]
    
    for line in r.text.split('\n')[2:-1]:
        word, sim = line.split('\t')
        if pos in word:
            neighbors.append(word)
    if 'VERB' == pos:
        return neighbors[1]
    return neighbors[0]

In [255]:
for word in s.split(' '):
    w = morph.parse(word)[0]
    normal_form = w.normal_form
    
    modified_word = w.word
    if w.tag.POS in ['INFN', 'VERB']:
        sim = api_neighbor(MODEL, normal_form, FORMAT)
        if sim != 'none':
            modified_word = morph.parse(sim.split('_')[0])[0].inflect({w.tag.tense, w.tag.mood}).word
    elif w.tag.POS == 'NOUN':
        if 'Name' in w.tag or 'Geox' in w.tag or 'Orgn' in w.tag:
            pass
        else:
            sim = api_neighbor(MODEL, normal_form, FORMAT)
            if sim != 'none':
                modified_word = morph.parse(sim.split('_')[0])[0].inflect({w.tag.case,w.tag.number}).word
    print(w.word, modified_word)

в в
москве москве
в в
воскресенье воскресение
днём неделей
начнётся возобновлюсь
сильный сильный
снегопад метель
его его
принесёт принешу
циклон антициклон
сара сара
сообщил извещали
риа риа
новости известия
ведущий ведущий
сотрудник руководитель
центра центра
погоды ненастья
фобос фобос
евгений евгений
тишковец тишковец


In [242]:
word = morph.parse('обвинить')[0]

In [243]:
word
#.animacy
#.inflect({'gent'})

Parse(word='обвинить', tag=OpencorporaTag('INFN,perf,tran'), normal_form='обвинить', score=1.0, methods_stack=((DictionaryAnalyzer(), 'обвинить', 754, 0),))

In [230]:
word.tag.tense

'past'

In [232]:
morph.parse('гадать')[0].inflect({word.tag.tense, word.tag.mood})

Parse(word='гадали', tag=OpencorporaTag('VERB,impf,intr plur,past,indc'), normal_form='гадать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'гадали', 15, 10),))

['взорваться_VERB\t0.6680192947387695', 'взрываться_VERB\t0.6318215131759644', 'детонация_NOUN\t0.6154584884643555', 'взрывной_ADJ\t0.6049678921699524', 'бомба_NOUN\t0.5874749422073364', 'катастрофа_NOUN\t0.5802870392799377', 'взрывать_VERB\t0.5706577301025391', 'землетрясение_NOUN\t0.5647920966148376', 'взорвать_VERB\t0.5582807064056396', 'воспламенений_NOUN\t0.5536834597587585', '']


'детонация'

In [13]:
def api_similarity(m, w1, w2):
    url = '/'.join(['https://rusvectores.org', m, w1 + '__' + w2, 'api', 'similarity/'])
    r = requests.get(url, stream=True)
    print(r.text)
    return r.text.split('\t')[0]

In [14]:
print(api_similarity(MODEL, WORD, 'мех_NOUN'))

0.10096654295921326	печень_NOUN	мех_NOUN	ruscorpora_upos_cbow_300_20_2019
0.10096654295921326
